In [2]:
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 462 kB 31.1 MB/s eta 0:00:01
     |████████████████████████████████| 190 kB 133.1 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 122.3 MB/s eta 0:00:01
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
     |████████████████████████████████| 213 kB 137.4 MB/s eta 0:00:01
     |████████████████████████████████| 143 kB 131.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 128.2 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 173 kB/s s eta 0:00:01
     |████████████████████████████████| 35.0 MB 119.2 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 138.9 MB/s eta 0:00:01
     |████████████████████████████████| 110 kB 115.9 MB/s eta 0:00:01
     |████████████████████████████████| 701 kB 82.3 MB/s eta 0:00:01
     |████████████████████████████████| 12.2 MB 107.1 MB/s eta 0:00:01
     |████████████████████

In [9]:
%pip install humanize

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 109 kB 35.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import datasets
import os
import humanize

PATH = '/fsx/mistobaan/datasets/S2ORC_ver2'

assert os.path.exists(PATH), PATH

# dataset = datasets.load_from_disk(PATH, keep_in_memory=True, streaming=True)

In [11]:
!git clone https://github.com/mistobaan/tokenizers.git --depth 1

Cloning into 'tokenizers'...
remote: Enumerating objects: 403, done.
remote: Counting objects: 100% (403/403), done.
remote: Compressing objects: 100% (367/367), done.
remote: Total 403 (delta 39), reused 178 (delta 17), pack-reused 0
Receiving objects: 100% (403/403), 687.80 KiB | 7.73 MiB/s, done.
Resolving deltas: 100% (39/39), done.
hint: The '/fsx/home-mistobaan/workspace/aws-hpc-ray-slurm-adventures/notebooks/tokenizers/.git/hooks/post-checkout' hook was ignored because it's not set as executable.
hint: You can disable this warning with `git config advice.ignoredHook false`.


In [3]:
from datasets import load_dataset
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

dataset = datasets.load_from_disk(PATH, streaming=True)

def encode(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length')

dataset = dataset.map(encode, batched=True)
next(iter(dataset))

ModuleNotFoundError: No module named 'tokenizers'

[Tokenizers](https://docs.rs/tokenizers/latest/tokenizers/)

In [4]:
from collections.abc import Mapping
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union

# Streaming

```python
dataset = load_dataset("datasetname", split='train', streaming=True) -> IterableDataset
```
# An IterableDataset is useful for iterative jobs like training a model.

# Shuffling

```python
shuffled_dataset = dataset.shuffle(seed=42, buffer_size=10_000)
shuffled_dataset.set_epoch(epoch) # to (re)-shuffle the dataset at each epoch with a different seed
```



## data access

```python
IterableDataset.take(n) # returns the first n examples in a dataset:
IterableDataset.skip(n) # skips the first n examples in a dataset:
```

```python
# multilingual_dataset = interleave_datasets([en_dataset, fr_dataset]) # interleaves the examples of the datasets
from datasets import interleave_datasets
multilingual_dataset_with_oversampling = interleave_datasets([en_dataset, fr_dataset], probabilities=[0.8, 0.2], seed=42)

## Column manipulation
- IterableDataset.rename_column(old_name, new_name) # renames a column

## Filtering
```python
start_with_ar = dataset.filter(lambda example: example['text'].startswith('Ar'))
```



In [27]:
# !cat $PATH/dataset_info.json

In [28]:
humanize.number.intword(len(dataset))

'11.7 million'

In [29]:
doc = next(iter(dataset))

In [ ]:
# set the datasets cache
%env HF_DATASETS_CACHE = /fsx/mistobaan/datasets_cache

In [ ]:
# useuful snippet to load a dataset from a custom path
from datasets import load_dataset
dataset = load_dataset('LOADING_SCRIPT', cache_dir="PATH/TO/MY/CACHE/DIR")
from datasets import load_metric
metric = load_metric('glue', 'mrpc', cache_dir="MY/CACHE/DIRECTORY")
dataset.cleanup_cache_files()

# restart from scratch the transformation
updated_dataset = small_dataset.map(add_prefix, load_from_cache_file=False)

from datasets import disable_caching
disable_caching()

from datasets import load_metric
metric = load_metric('glue', 'mrpc', keep_in_memory=True)

Set `datasets.config.IN_MEMORY_MAX_SIZE` to a nonzero value (in bytes) that fits in your RAM memory.

Set the environment variable `HF_DATASETS_IN_MEMORY_MAX_SIZE` to a nonzero value. Note that the first method takes higher precedence.


# Tokenization

In [2]:
dataset.set_format(type="torch", 
                   columns=["input_ids", "token_type_ids", "attention_mask", "labels"])
dataset.set_transform(transforms) # applies the transfomration on the fly
dataset = dataset.map(tokenization, batched=True)

NameError: name 'dataset' is not defined

In [7]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForMaskedLM, DataCollatorForLanguageModeling
from tqdm import tqdm
dataset = dataset.with_format("torch")
dataloader = DataLoader(dataset, collate_fn=DataCollatorForLanguageModeling(tokenizer))
device = 'cuda' if torch.cuda.is_available() else 'cpu' 
model = AutoModelForMaskedLM.from_pretrained("distilbert-base-uncased")
model.train().to(device)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)
for epoch in range(3):
    dataset.set_epoch(epoch)
    for i, batch in enumerate(tqdm(dataloader, total=5)):
        if i == 5:
            break
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
            print(f"loss: {loss}")

ModuleNotFoundError: No module named 'torch'